In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from xgbimputer import XGBImputer

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = 100

In [3]:
df = pd.read_csv('./datasets/House-Prices-Advanced-Regression.csv')
df = df.drop(columns='Id')
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,

In [4]:
def get_missing_data(df):
    df_missing_data = pd.DataFrame(df.isna().sum().loc[df.isna().sum() > 0], columns=['missing_data_count'])
    df_missing_data['missing_data_type'] = df.dtypes
    df_missing_data['missing_data_percentage'] = df_missing_data['missing_data_count'] / len(df)
    df_missing_data = df_missing_data.sort_values(by='missing_data_percentage', ascending=False)
    return df_missing_data.style.format({'missing_data_percentage': '{:,.2%}'})

In [5]:
get_missing_data(df)

,missing_data_count,missing_data_type,missing_data_percentage
PoolQC,1453,object,99.52%
MiscFeature,1406,object,96.30%
Alley,1369,object,93.77%
Fence,1179,object,80.75%
FireplaceQu,690,object,47.26%
LotFrontage,259,float64,17.74%
GarageType,81,object,5.55%
GarageYrBlt,81,float64,5.55%
GarageFinish,81,object,5.55%
GarageQual,81,object,5.55%


In [6]:
df.loc[:, ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType']] = df.loc[:, ['PoolQC', 'MiscFeature', 'Alley', 'Fence', 'FireplaceQu', 'GarageType', 'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond', 'BsmtExposure', 'BsmtFinType2', 'BsmtFinType1', 'BsmtCond', 'BsmtQual', 'MasVnrType']].fillna('NA')
df.loc[:, 'MasVnrArea'] = df.loc[:, 'MasVnrArea'].fillna(0)
df = df.dropna()

In [7]:
get_missing_data(df)

,missing_data_count,missing_data_type,missing_data_percentage
MSSubClass,nan,int64,nan%
MSZoning,nan,object,nan%
LotFrontage,nan,float64,nan%
LotArea,nan,int64,nan%
Street,nan,object,nan%
Alley,nan,object,nan%
LotShape,nan,object,nan%
LandContour,nan,object,nan%
Utilities,nan,object,nan%
LotConfig,nan,object,nan%


In [8]:
df = df.reset_index(drop=True)
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NA,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NA,NA,NA,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NA,NA,NA,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,60,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,Ex,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal,175000
1196,20,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,TA,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NA,MnPrv,NA,0,2,2010,WD,Normal,210000
1197,70,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,Ex,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1198,20,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,GasA,Gd,Y,FuseA,1078,0,0,1078,1,0,1,0,2,1,Gd,5,Typ,0,NA,Attchd,1950.0,U

In [9]:
categorical_features = pd.concat([pd.Series(['MSSubClass']), pd.Series(df.select_dtypes(include='object').columns)], ignore_index=True)
categorical_features

0        MSSubClass
1          MSZoning
2            Street
3             Alley
4          LotShape
5       LandContour
6         Utilities
7         LotConfig
8         LandSlope
9      Neighborhood
10       Condition1
11       Condition2
12         BldgType
13       HouseStyle
14        RoofStyle
15         RoofMatl
16      Exterior1st
17      Exterior2nd
18       MasVnrType
19        ExterQual
20        ExterCond
21       Foundation
22         BsmtQual
23         BsmtCond
24     BsmtExposure
25     BsmtFinType1
26     BsmtFinType2
27          Heating
28        HeatingQC
29       CentralAir
30       Electrical
31      KitchenQual
32       Functional
33      FireplaceQu
34       GarageType
35      GarageYrBlt
36     GarageFinish
37       GarageQual
38       GarageCond
39       PavedDrive
40           PoolQC
41            Fence
42      MiscFeature
43         SaleType
44    SaleCondition
dtype: object

In [10]:
categorical_features_index = []
for column in categorical_features:
    categorical_features_index.append(df.columns.get_loc(column))

In [11]:
numerical_features = pd.Series(df.select_dtypes(exclude='object').drop(columns=['MSSubClass', 'SalePrice']).columns)
numerical_features

0       LotFrontage
1           LotArea
2       OverallQual
3       OverallCond
4         YearBuilt
5      YearRemodAdd
6        MasVnrArea
7        BsmtFinSF1
8        BsmtFinSF2
9         BsmtUnfSF
10      TotalBsmtSF
11         1stFlrSF
12         2ndFlrSF
13     LowQualFinSF
14        GrLivArea
15     BsmtFullBath
16     BsmtHalfBath
17         FullBath
18         HalfBath
19     BedroomAbvGr
20     KitchenAbvGr
21     TotRmsAbvGrd
22       Fireplaces
23       GarageCars
24       GarageArea
25       WoodDeckSF
26      OpenPorchSF
27    EnclosedPorch
28        3SsnPorch
29      ScreenPorch
30         PoolArea
31          MiscVal
32           MoSold
33           YrSold
dtype: object

In [12]:
np.random.seed(42)

In [13]:
subsample_numerical_features = numerical_features.sample(frac=np.random.uniform(high=0.6))
subsample_numerical_features

15     BsmtFullBath
19     BedroomAbvGr
27    EnclosedPorch
26      OpenPorchSF
8        BsmtFinSF2
24       GarageArea
21     TotRmsAbvGrd
12         2ndFlrSF
dtype: object

In [14]:
subsample_categorical_features = categorical_features.sample(frac=np.random.uniform(high=0.6))
subsample_categorical_features

0     MSSubClass
21    Foundation
28     HeatingQC
10    Condition1
dtype: object

In [15]:
df_filled_with_nans = df.copy()

In [16]:
for column in pd.concat([subsample_numerical_features, subsample_categorical_features]):
    nan_percentage = np.random.uniform(low=0.1, high=0.5)
    print(f'Column: {column} / Missing percentage: {nan_percentage}')
    column_idx = df_filled_with_nans.columns.get_loc(column)
    df_filled_with_nans.iloc[df[column].sample(frac=nan_percentage).index, column_idx] = np.nan

Column: BsmtFullBath / Missing percentage: 0.4579309401710595
Column: BedroomAbvGr / Missing percentage: 0.4602289204945543
Column: EnclosedPorch / Missing percentage: 0.3198464174875698
Column: OpenPorchSF / Missing percentage: 0.27420305018887625
Column: BsmtFinSF2 / Missing percentage: 0.3944959240535333
Column: GarageArea / Missing percentage: 0.3281147754925562
Column: TotRmsAbvGrd / Missing percentage: 0.10961649494657513
Column: 2ndFlrSF / Missing percentage: 0.3001162074712719
Column: MSSubClass / Missing percentage: 0.48777658765290277
Column: Foundation / Missing percentage: 0.2190694862371383
Column: HeatingQC / Missing percentage: 0.11966340944857104
Column: Condition1 / Missing percentage: 0.27546215254014983


In [17]:
get_missing_data(df)

,missing_data_count,missing_data_type,missing_data_percentage
MSSubClass,nan,int64,nan%
MSZoning,nan,object,nan%
LotFrontage,nan,float64,nan%
LotArea,nan,int64,nan%
Street,nan,object,nan%
Alley,nan,object,nan%
LotShape,nan,object,nan%
LandContour,nan,object,nan%
Utilities,nan,object,nan%
LotConfig,nan,object,nan%


In [18]:
get_missing_data(df_filled_with_nans)

,missing_data_count,missing_data_type,missing_data_percentage
MSSubClass,585,float64,48.75%
BedroomAbvGr,552,float64,46.00%
BsmtFullBath,550,float64,45.83%
BsmtFinSF2,473,float64,39.42%
GarageArea,394,float64,32.83%
EnclosedPorch,384,float64,32.00%
2ndFlrSF,360,float64,30.00%
Condition1,331,object,27.58%
OpenPorchSF,329,float64,27.42%
Foundation,263,object,21.92%


In [19]:
X = np.array(df_filled_with_nans)
X

array([[nan, 'RL', 65.0, ..., 'WD', 'Normal', 208500],
       [20.0, 'RL', 80.0, ..., 'WD', 'Normal', 181500],
       [nan, 'RL', 68.0, ..., 'WD', 'Normal', 223500],
       ...,
       [70.0, 'RL', 66.0, ..., 'WD', 'Normal', 266500],
       [20.0, 'RL', 68.0, ..., 'WD', 'Normal', 142125],
       [20.0, 'RL', 75.0, ..., 'WD', 'Normal', 147500]], dtype=object)

In [20]:
X.shape

(1200, 80)

In [21]:
y = X[:,-1]
X = X[:,0:-1]

In [22]:
X.shape

(1200, 79)

In [23]:
xgbimputer = XGBImputer(categorical_features_index=categorical_features_index, replace_categorical_values_back=True)

In [24]:
X = xgbimputer.fit_transform(X)

XGBImputer - Epoch: 1 | Categorical gamma: inf/330.75 | Numerical gamma: inf/nan
XGBImputer - Epoch: 2 | Categorical gamma: 330.75/15.25 | Numerical gamma: nan/0.0055396219
XGBImputer - Epoch: 3 | Categorical gamma: 15.25/11.25 | Numerical gamma: 0.0055396219/0.0034608729
XGBImputer - Epoch: 4 | Categorical gamma: 11.25/9.25 | Numerical gamma: 0.0034608729/0.0035596476
XGBImputer - Epoch: 5 | Categorical gamma: 9.25/11.75 | Numerical gamma: 0.0035596476/0.0033699971
XGBImputer - Epoch: 6 | Categorical gamma: 11.75/12. | Numerical gamma: 0.0033699971/0.004453066


In [25]:
type(X)

numpy.ndarray

In [26]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78
0,60.0,RL,65.0,8450,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,-0.174779,150,856,GasA,Ex,Y,SBrkr,856,854.0,0,1710,1.0,0,2,1,3.0,1,Gd,8.0,Typ,0,NA,Attchd,2003.0,RFn,2,548.0,TA,TA,Y,0,61.0,0.0,0,0,0,NA,NA,NA,0,2,2008,WD,Normal
1,20.0,RL,80.0,9600,Pave,NA,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,-1.57131,284,1262,GasA,Ex,Y,SBrkr,1262,-3.905657,0,1262,0.0,1,2,0,3.0,1,TA,6.0,Typ,1,TA,Attchd,1976.0,RFn,2,460.0,TA,TA,Y,298,40.798676,0.0,0,0,0,NA,NA,NA,0,5,2007,WD,Normal
2,60.0,RL,68.0,11250,Pave,NA,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0.245691,434,920,GasA,Ex,Y,SBrkr,920,884.826843,0,1786,0.197372,0,2,1,3.0,1,Gd,6.0,Typ,1,TA,Attchd,2001.0,RFn,2,608.0,TA,TA,Y,0,42.0,22.347898,0,0,0,NA,NA,NA,0,9,2008,WD,Normal
3,70.0,RL,60.0,9550,Pave,NA,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0.0,540,756,GasA,Gd,Y,SBrkr,961,912.673584,0,1717,0.733151,0,1,0,3.304353,1,Gd,7.0,Typ,1,Gd,Detchd,1998.0,Unf,3,642.0,TA,TA,Y,0,35.0,82.090591,0,0,0,NA,NA,NA,0,2,2006,WD,Abnorml
4,60.0,RL,84.0,14260,Pave,NA,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0.0,490,1145,GasA,Ex,Y,SBrkr,1145,874.679688,0,2198,1.0,0,2,1,3.528136,1,Gd,8.179894,Typ,1,TA,Attchd,2000.0,RFn,3,674.172974,TA,TA,Y,192,84.0,-14.324702,0,0,0,NA,NA,NA,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,60.0,RL,62.0,7917,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,-0.152759,953,953,GasA,Ex,Y,SBrkr,953,760.095093,0,1647,-0.201907,0,2,1,3.000851,1,TA,7.0,Typ,1,TA,Attchd,1999.0,RFn,2,462.427124,TA,TA,Y,0,40.0,0.0,0,0,0,NA,NA,NA,0,8,2007,WD,Normal
1196,20.0,RL,85.0,13175,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163.0,589,1542,GasA,TA,Y,SBrkr,2073,-24.414165,0,2073,1.0,0,2,0,2.460662,1,TA,7.0,Min1,2,TA,Attchd,1978.0,Unf,2,527.355408,TA,TA,Y,349,0.0,56.706856,0,0,0,NA,MnPrv,NA,0,2,2010,WD,Normal
1197,70.0,RL,66.0,9042,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,2.059961,877,1152,GasA,Ex,Y,SBrkr,1188,1152.0,0,2340,0.0,0,2,0,4.0,1,Gd,9.0,Typ,2,Gd,Attchd,1941.0,RFn,1,252.0,TA,TA,Y,0,145.763062,90.963364,0,0,0,NA,GdPrv,Shed,2500,5,2010,WD,Normal
1198,20.0,RL,68.0,9717,Pave,NA,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029.0,0,1078,GasA,Gd,Y,FuseA,1078,0.0,0,1078,1.0,0,1,0,2.0,1,Gd,5.0,Typ,0,NA,Attchd,1950.0,Unf,1,240.0,TA,TA,Y,366,0.0,112.0,0,0,0,NA,NA,NA,0,4,2010,WD,Normal
